# Inference Demo

Load a checkpoint and run inference on new microscope images. Visualize heatmaps and predicted classes.

In [ ]:
# Imports
import os
import matplotlib.pyplot as plt
from src.inference.predict import predict_image
from PIL import Image

In [ ]:
# Set paths
model_path = os.path.join('..','models','best_model.pth')
image_dir = os.path.join('..','data','test','200°C')
image_path = os.path.join(image_dir, os.listdir(image_dir)[0])

In [ ]:
# Run prediction and plot results
probs = predict_image(image_path, model_path)
classes = list(probs.keys())
values = list(probs.values())
plt.figure(figsize=(6,4))
plt.bar(classes, values)
plt.xlabel('Class')
plt.ylabel('Probability')
plt.title('Prediction Probabilities')
plt.show()

img = Image.open(image_path)
plt.imshow(img)
plt.axis('off')
plt.show()